In [197]:
import os
import time
import json
import requests
import tempfile
import numpy as np
import pandas as pd

import boto3
import sagemaker
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep, CreateModelStep, ProcessingStep

from sagemaker.estimator import Estimator
from sagemaker import get_execution_role

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import ParameterString
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
from sagemaker.workflow.model_step import ModelStep
from sagemaker.sklearn.processing import SKLearnProcessor

In [198]:
region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)
sagemaker_boto_client = boto_session.client("sagemaker")

# sagemaker_session = sagemaker.session.Session()
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_boto_client
)

account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]
# region = sagemaker_session.boto_session.region_name
s3_client = boto3.client("s3", region_name=region)
sm_client = boto3.client("sagemaker", region_name=region)

role = get_execution_role()

In [178]:
# S3 prefix
bucket = "artwork-content-trial-bucket"
prefix="mch-artwork-content"
train_data_dir_prefix="data"
pipeline_dir_prefix="pipeline-data"
training_input_prefix = "content_ovr_anonymised.csv"
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="Approved"
)


In [179]:
training_input = f"s3://{bucket}/{train_data_dir_prefix}/{training_input_prefix}"

In [180]:
train_image_uri = '791574662255.dkr.ecr.us-east-1.amazonaws.com/artwork-content-repo:latest'
estimator_output_path = f"s3://{bucket}/{pipeline_dir_prefix}/"

In [181]:

content_estimator = Estimator(
    image_uri=train_image_uri,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    entry_point='inference.py'
    output_path=estimator_output_path,
    role=role,
)

In [182]:
step_train = TrainingStep(
    name="training",
    estimator=content_estimator,
    inputs=training_input,
)

In [183]:
mdl_name="artwork_content_model"

model = Model(
    name=mdl_name,
    image_uri=content_estimator.training_image_uri(),
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=PipelineSession(),
    role=role,
)

In [184]:
pipeline_session = PipelineSession()

pipeline_model = PipelineModel(
        models=[model],
        role=role,
        sagemaker_session=pipeline_session,
)

In [195]:
# timestamp = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
# mpg_name = f"artwork-content-{timestamp}"
mpg_name = "MCH-Content-Models"

register_model_step_args = pipeline_model.register(
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    model_package_group_name=mpg_name,
    approval_status=model_approval_status
)

/opt/conda/lib/python3.7/site-packages/sagemaker/workflow/pipeline_context.py:261: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  UserWarning,


AttributeError: 'NoneType' object has no attribute 'need_runtime_repack'

In [204]:
# from datetime import datetime

# endpoint_name = f"DEMO-{datetime.utcnow():%Y-%m-%d-%H%M}"
# print("EndpointName =", endpoint_name)

# initial_instance_count=<integer>
# # initial_instance_count=1 # Example

# instance_type='<instance-type>'
# # instance_type='ml.m4.xlarge' # Example

# model.deploy(
#     initial_instance_count=initial_instance_count,
#     instance_type=instance_type,
#     endpoint_name=endpoint_name
# )

In [212]:
# model_package_arn = sm_client.list_model_packages(ModelPackageGroupName=mpg_name)['ModelPackageSummaryList'][0]['ModelPackageArn']

# from sagemaker import ModelPackage
# from time import gmtime, strftime

# # model_package_arn = 'arn:aws:sagemaker:us-east-2:12345678901:model-package/modeltest/1'
# model = ModelPackage(role=role, 
#                      model_package_arn=model_package_arn, 
#                      sagemaker_session=sagemaker_session)
# model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

-

KeyboardInterrupt: 

In [ ]:
model_name = "artwork-content-model" 
step_model_registration = ModelStep(
   name=model_name,
   step_args=register_model_step_args,
)

In [ ]:
step_model_create = ModelStep(
   name=f"{model_name}-creation",
   step_args=model.create(instance_type="ml.m5.xlarge"),
)

In [ ]:
s3_client.upload_file(Filename="deploy.py", Bucket=bucket, Key="code/deploy.py")
deploy_script_uri = f"s3://{bucket}/code/deploy.py"

deployment_processor = SKLearnProcessor(
    framework_version="1.0-1",
    role=role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name=f"{prefix}-deploy",
    sagemaker_session=sagemaker_session,
)

deploy_step = ProcessingStep(
    name="DeployModel",
    processor=deployment_processor,
    job_arguments=[
        "--model-name",
        step_model_create.properties.ModelName,
        "--region",
        region,
        "--endpoint-instance-type",
        "ml.m4.xlarge",
        "--endpoint-name",
        "mch-artwork-content-endpoint-1",
    ],
    code=deploy_script_uri,
)

In [ ]:
pipeline_name = f"artwork-content-pipeline-demo"

pipeline_steps = [step_train, step_model_create, step_model_registration, deploy_step]

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[model_approval_status],
    steps=pipeline_steps,
)

In [ ]:
# json.loads(pipeline.definition())

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
execution = pipeline.start()
execution.wait()
execution.describe()

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)